In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import DataPreprocessing
import os
import sys

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# os.environ['PYSPARK_PYTHON']= 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310'
# os.environ['PYSPARK_DRIVER_PYTHON']= 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\python.exe'
# os.environ['SPARK_HOME'] = 'F:\\spark-3.4.0-bin-hadoop3\\spark-3.4.0-bin-hadoop3'
# os.environ['HADOOP_HOME'] = 'F:\\spark-3.4.0-bin-hadoop3\\spark-3.4.0-bin-hadoop3\\hadoop'


### Spark session

In [2]:
sc = SparkContext(master='local')
spark=SparkSession.builder.appName('DataPreprocessing').getOrCreate()


### Splitting the data 

In [3]:
# DataPreprocessing.split_data()

### Replace delimiters

In [10]:
# DataPreprocessing.replace_delimiters(delimiter='|')
# DataPreprocessing.replace_delimiters(delimiter='|',kind='train')
# DataPreprocessing.replace_delimiters(delimiter='|',kind='val')
# DataPreprocessing.replace_delimiters(delimiter='|',kind='test')

### Reading the data

In [4]:
# useless_cols = ['Scraped Time', 'Privacy Policy', 'Currency', 'Editors Choice']
df = DataPreprocessing.read_data(spark,kind='val', features='Categorical', encode=True, drop_cols=[])
df.show(5)

+--------------+------------+--------------+--------------+----------+--------------+----------+---------------------+------------------+-----------------------+---------------------+--------------+------------------+--------------------+--------------------+------------------+----------------------+--------------------+------------------+
|App Name_index|App Id_index|Category_index|Installs_index|Free_index|Currency_index|Size_index|Minimum Android_index|Developer Id_index|Developer Website_index|Developer Email_index|Released_index|Last Updated_index|Content Rating_index|Privacy Policy_index|Ad Supported_index|In App Purchases_index|Editors Choice_index|Scraped Time_index|
+--------------+------------+--------------+--------------+----------+--------------+----------+---------------------+------------------+-----------------------+---------------------+--------------+------------------+--------------------+--------------------+------------------+----------------------+---------------

### Get info about the dataset

In [5]:
DataPreprocessing.get_info(df)

Number of rows: 462589, Number of columns: 19
Available features: ['App Name_index', 'App Id_index', 'Category_index', 'Installs_index', 'Free_index', 'Currency_index', 'Size_index', 'Minimum Android_index', 'Developer Id_index', 'Developer Website_index', 'Developer Email_index', 'Released_index', 'Last Updated_index', 'Content Rating_index', 'Privacy Policy_index', 'Ad Supported_index', 'In App Purchases_index', 'Editors Choice_index', 'Scraped Time_index']
+-------+------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+---------------------+------------------+-----------------------+---------------------+------------------+------------------+--------------------+--------------------+-------------------+----------------------+--------------------+------------------+
|summary|    App Name_index|      App Id_index|    Category_index|    Installs_index|          Free_index|      Currency_index|        Size

### Dealing with missing values

In [6]:
df_copy= DataPreprocessing.missing_values(df)

Total Number of rows : 462589
Number of rows with missing values: 0
Number of rows after dropping: 462589


### Outliers

In [ ]:
DataPreprocessing.detect_outliers(df, 'Rating')